In [ ]:
%env
ETL_CONF_BASE_DIR=/home/jovyan/examples/iso20022

In [ ]:
{
  "type": "XMLExtract",
  "name": "load pain.001.001.03",
  "environments": [
    "production",
    "test"
  ],
  "inputURI": ${ETL_CONF_BASE_DIR}"/pain.001.001.03.xml",
  "schemaURI": ${ETL_CONF_BASE_DIR}"/pain.001.001.03.json",
  "xsdURI": ${ETL_CONF_BASE_DIR}"/pain.001.001.03.xsd",
  "outputView": "pain_001_001_03"
}


In [ ]:
%printschema
pain_001_001_03

In [ ]:
%sql outputView=GrpHdr
SELECT 
  CstmrCdtTrfInitn.GrpHdr.CreDtTm AS CreDtTm
  ,CstmrCdtTrfInitn.GrpHdr.CtrlSum AS CtrlSum
  ,CstmrCdtTrfInitn.GrpHdr.InitgPty.Nm AS InitgPtyNm
  ,CstmrCdtTrfInitn.GrpHdr.MsgId AS MsgId
  ,CstmrCdtTrfInitn.GrpHdr.NbOfTxs AS NbOfTxs
FROM pain_001_001_03

In [ ]:
%sql outputView=PmtInf
SELECT 
  CstmrCdtTrfInitn.GrpHdr.MsgId MsgId
  ,CstmrCdtTrfInitn.PmtInf.ChrgBr AS ChrgBr
  ,CstmrCdtTrfInitn.PmtInf.CtrlSum AS CtrlSum
  ,CstmrCdtTrfInitn.PmtInf.Dbtr.Nm AS DbtrNm
  ,CstmrCdtTrfInitn.PmtInf.DbtrAcct.Id.IBAN AS DbtrAcctIdIBAN
  ,CstmrCdtTrfInitn.PmtInf.DbtrAgt.FinInstnId.BIC AS DbtrAgtFinInstnIdBIC
  ,CstmrCdtTrfInitn.PmtInf.NbOfTxs AS NbOfTxs
  ,CstmrCdtTrfInitn.PmtInf.PmtInfId AS PmtInfId
  ,CstmrCdtTrfInitn.PmtInf.PmtMtd AS PmtMtd
  ,CstmrCdtTrfInitn.PmtInf.PmtTpInf.SvcLvl.Cd AS PmtTpInfSvcLvlCd
  ,CstmrCdtTrfInitn.PmtInf.ReqdExctnDt AS ReqdExctnDt
FROM pain_001_001_03

In [ ]:
%sql outputView=CdtTrfTxInf
SELECT 
  PmtInfId
  ,CdtTrfTxInf.Amt.InstdAmt._Ccy AS AmtInstdAmtCcy
  ,CdtTrfTxInf.Amt.InstdAmt._VALUE AS AmtInstdAmt
  ,CdtTrfTxInf.Cdtr.Nm AS CdtrNm
  ,CdtTrfTxInf.CdtrAcct.Id.IBAN AS CdtrAcctIdIBAN
  ,CdtTrfTxInf.CdtrAgt.FinInstnId.BIC AS CdtrAgtFinInstnIdBIC
  ,CdtTrfTxInf.PmtId.EndToEndId AS PmtIdEndToEndId
  ,CdtTrfTxInf.RmtInf.Ustrd AS RmtInfUstrd
FROM (
  SELECT 
    CstmrCdtTrfInitn.PmtInf.PmtInfId
    ,EXPLODE(CstmrCdtTrfInitn.PmtInf.CdtTrfTxInf) AS CdtTrfTxInf
  FROM pain_001_001_03
)

In [ ]:
%sqlvalidate truncate=1000
SELECT 
  NbOfTxs = CdtTrfTxInfCount AND CtrlSum = CdtTrfTxInfSum AS valid 
  ,TO_JSON(
    NAMED_STRUCT(
      'NbOfTxs', NAMED_STRUCT(
        'expected', NbOfTxs,
        'actual', CdtTrfTxInfCount
      ),
      'CtrlSum', NAMED_STRUCT(
        'expected', CtrlSum,
        'actual', CdtTrfTxInfSum
      )      
    )
  ) AS message
FROM PmtInf
CROSS JOIN
(
  SELECT 
    COUNT(AmtInstdAmt) AS CdtTrfTxInfCount
    ,SUM(AmtInstdAmt) AS CdtTrfTxInfSum
  FROM CdtTrfTxInf
) CdtTrfTxInf

In [ ]:
%sql monospace=true leftAlign=true truncate=10000
SELECT 
  CONCAT('<?xml version="1.0" encoding="utf-8"?>', TO_XML(
    NAMED_STRUCT(
      'Document', NAMED_STRUCT(
        'CstmrCdtTrfInitn', NAMED_STRUCT(
          'GrpHdr', NAMED_STRUCT(
            'CreDtTm', GrpHdr.CreDtTm
            ,'CtrlSum', GrpHdr.CtrlSum
            ,'InitgPty', NAMED_STRUCT(
              'Nm', GrpHdr.InitgPtyNm
            )
            ,'MsgId', GrpHdr.MsgId
            ,'NbOfTxs', GrpHdr.NbOfTxs
          )
          ,'PmtInf', COLLECT_LIST(PmtInf.PmtInf)
        )
        ,'_xmlns', 'urn:iso:std:iso:20022:tech:xsd:pain.001.001.03'
        ,'_xsd', 'http://www.w3.org/2001/XMLSchema'
        ,'_xsi', 'http://www.w3.org/2001/XMLSchema-instance'
      )
    )
  )) AS xml
FROM GrpHdr
INNER JOIN (
  SELECT 
    MsgId
    ,NAMED_STRUCT(
      'ChrgBr', PmtInf.ChrgBr
      ,'CtrlSum', PmtInf.CtrlSum
      ,'Dbtr', NAMED_STRUCT(
        'Nm', PmtInf.DbtrNm
      )
      ,'DbtrAcct', NAMED_STRUCT(
        'Id', NAMED_STRUCT(
          'IBAN', PmtInf.DbtrAcctIdIBAN
        )
      )
      ,'DbtrAgt', NAMED_STRUCT(
        'FinInstnId', NAMED_STRUCT(
          'BIC', PmtInf.DbtrAgtFinInstnIdBIC
        )
      )        
      ,'NbOfTxs', PmtInf.NbOfTxs
      ,'PmtInfId', PmtInf.PmtInfId
      ,'PmtMtd', PmtInf.PmtMtd
      ,'PmtTpInf', NAMED_STRUCT(
        'SvcLvl', NAMED_STRUCT(
          'Cd', PmtInf.PmtTpInfSvcLvlCd
        )
      )    
      ,'ReqdExctnDt', PmtInf.ReqdExctnDt
      ,'CdtTrfTxInf', COLLECT_LIST(CdtTrfTxInf)
    ) AS PmtInf
  FROM PmtInf
  INNER JOIN (
    SELECT 
      PmtInfId
      ,NAMED_STRUCT(
        'Amt', NAMED_STRUCT(
          'InstdAmt', NAMED_STRUCT(
            '_VALUE', CdtTrfTxInf.AmtInstdAmt,
            '_Ccy', CdtTrfTxInf.AmtInstdAmtCcy
          )
        )
        ,'Cdtr', NAMED_STRUCT(
          'Nm', CdtTrfTxInf.CdtrNm
        )
        ,'CdtrAcct', NAMED_STRUCT(
          'Id', NAMED_STRUCT(
            'IBAN', CdtTrfTxInf.CdtrAcctIdIBAN
          )
        )
        ,'CdtrAgt', NAMED_STRUCT(
          'FinInstnId', NAMED_STRUCT(
            'BIC', CdtTrfTxInf.CdtrAgtFinInstnIdBIC
          )
        )
        ,'PmtId', NAMED_STRUCT(
          'EndToEndId', CdtTrfTxInf.PmtIdEndToEndId
        )  
        ,'RmtInf', NAMED_STRUCT(
          'Ustrd', CdtTrfTxInf.RmtInfUstrd
        )               
      ) AS CdtTrfTxInf
    FROM CdtTrfTxInf
  ) CdtTrfTxInf ON PmtInf.PmtInfId = CdtTrfTxInf.PmtInfId

  GROUP BY PmtInf.MsgId, PmtInf.ChrgBr, PmtInf.CtrlSum, PmtInf.DbtrNm, PmtInf.DbtrAcctIdIBAN, PmtInf.DbtrAgtFinInstnIdBIC, PmtInf.NbOfTxs, PmtInf.PmtInfId, PmtInf.PmtMtd, PmtInf.PmtTpInfSvcLvlCd, PmtInf.ReqdExctnDt
) PmtInf ON GrpHdr.MsgId = PmtInf.MsgId

GROUP BY GrpHdr.CreDtTm, GrpHdr.CtrlSum, GrpHdr.InitgPtyNm, GrpHdr.MsgId, GrpHdr.NbOfTxs